# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [43]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import re
import pandas as pd

from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
stopwords = stopwords.words("english")

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [4]:
convention_data = []

# fill the above list up with items that are themselves lists. The 
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT party, text FROM conventions
                            where party != "Other"
                            ''')

for row in query_results :
    # store the results in convention_data
    # remove this and replace with your code
    party = row[0]
    speech = row[1]
    convention_data.append([speech, party])


In [5]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right. 

In [6]:
random.choices(convention_data,k=5)

[['And now, we’ll hear from Kristin Urquiza.', 'Democratic'],
 ['In my husband, you have a president who will not stop fighting for you and your families. I see how hard he works each day and night, and despite the unprecedented attacks from the media and opposition, he will not give up. In fact, if you tell him he cannot be done, he just works harder. Donald is a husband who supports me in all that I do. He has built an administration with an unprecedented number of women in leadership roles, and has fostered an environment where the American people are always the priority. He welcomes different points of view and encourage thinking outside of the box. I know I speak for my husband and the family when I say we are so grateful that you have trusted him to be your president, and we will be honored to serve this incredible country for four more years. As you have heard this evening, I don’t want to use this precious time attacking the other side, because as we saw last week, that kind of

It'll be useful for us to have a large sample size than 2024 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html). 

In [7]:
conv_sent_data = []

for speech, party in convention_data :
    first = nltk.sent_tokenize(speech)[0]
    conv_sent_data.append([first, party])


Again, let's look at some random entries. 

In [8]:
random.choices(conv_sent_data,k=5)

[['So today, six months after it began, the nation is still unprepared.',
  'Democratic'],
 ['How long was your problem.', 'Republican'],
 ['Good evening.', 'Republican'],
 ['These women and the generations that followed worked to make democracy and opportunity real in the lives of all of us who followed.',
  'Democratic'],
 ['The child who knows that black lives matter.', 'Democratic']]

Now it's time for our final cleaning before modeling. Go through `conv_sent_data` and take the following steps: 

1. Tokenize on whitespace
1. Remove punctuation
1. Remove tokens that fail the `isalpha` test
1. Remove stopwords
1. Casefold to lowercase
1. Join the remaining tokens into a string


In [9]:
clean_conv_sent_data = [] # list of tuples (sentence, party), with sentence cleaned

for idx, sent_party in conv_sent_data :
    tokens = word_tokenize(idx)
    tokens = [token for token in tokens if token not in punctuation]
    tokens = [token for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token.lower() not in stopwords]
    tokens = [token.lower() for token in tokens]
    sent = ' '.join(tokens)
    clean_conv_sent_data.append([sent, sent_party])
    
    

random.choices(clean_conv_sent_data,k=5)

[['allegiance fidelity', 'Republican'],
 ['know typical former surgeon general speak convention', 'Democratic'],
 ['reverend kind know like lose family heart goes', 'Democratic'],
 ['indiana', 'Democratic'],
 ['knows saving planet', 'Democratic']]

If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5. 

In [10]:
word_cutoff = 5

tokens = [w for t, p in clean_conv_sent_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 628 as features in the model.


In [11]:
def conv_features(text,feature_words) :
     ret_dict = dict()
     word = text.split()
     ret_dict = {word: True for word in word if word in feature_words}
     return(ret_dict)

In [12]:
assert(len(feature_words)>0)
assert(conv_features("obama was the president",feature_words)==
       {'obama':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [13]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [14]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [15]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.494


In [16]:
classifier.show_most_informative_features(25)

Most Informative Features
             enforcement = True           Republ : Democr =     27.5 : 1.0
                   votes = True           Democr : Republ =     21.6 : 1.0
                 climate = True           Democr : Republ =     17.3 : 1.0
                   media = True           Republ : Democr =     15.9 : 1.0
              appreciate = True           Republ : Democr =     14.0 : 1.0
                    drug = True           Republ : Democr =     10.3 : 1.0
                 special = True           Republ : Democr =     10.3 : 1.0
                   local = True           Republ : Democr =      9.9 : 1.0
                   elect = True           Democr : Republ =      9.6 : 1.0
                    land = True           Republ : Democr =      8.9 : 1.0
                  cities = True           Republ : Democr =      8.4 : 1.0
                citizens = True           Republ : Democr =      8.4 : 1.0
                    flag = True           Republ : Democr =      8.4 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

It seems that the words "encorcement", "media", "appreciate", "drug", "special" and "local" are the words that indicate to this model that a speech is done by a Republican. The words "votes", "climate", "elect", and "clean" are the words that the model finds most important for classifying a speech is done by a Democrat. There seem to be a lot more words that indicate a Republican speech over a Democrat speech than vice versa. 



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [18]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [19]:
cong_db.close()

In [20]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results :
    # store the results in convention_data
    # remove this and replace with your code
    candidate = row[0]
    party = row[1]
    tweet = row[2]
    tweet_data.append([tweet, party])


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [21]:
random.seed(20250602)

tweet_data_sample = random.choices(tweet_data,k=10)

In [32]:
def estimator(text):
    features = conv_features(str(text),feature_words) 
    est = classifier.classify(features)
    return est

In [40]:
for tweet, party in tweet_data_sample :
    estimated_party = estimator(tweet)
    
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: b"Enjoyed updating the Mount Ayr Chamber of Commerce on what I've been working on in Congress. #IA03 https://t.co/74iEFpwkL5"
Actual party is Republican and our classifer says Democratic.

Here's our (cleaned) tweet: b'Coming to you live from the Basalt town hall with Rep. Hamner https://t.co/Cstsx9NeP0'
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: b'A citizen was shot by Park Police officers. The authorities won\xe2\x80\x99t say why.\n\nThank you @washingtonpost @PostOpinions for pressing the FBI to deliver answers about the killing of Bijan Ghaisar. His family has been denied justice for far too long. https://t.co/nnXA2XWXam'
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: b'Sandy Hook Elementary School prepping to join the #Monroe Memorial Day Parade. https://t.co/cYiB3Le86K'
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: b"

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [41]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    party = party
    estimated_party = estimator(tweet)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [42]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 2431, 'Democratic': 1905}),
             'Democratic': defaultdict(int,
                         {'Republican': 3448, 'Democratic': 2218})})

In [49]:
df = pd.DataFrame(results)
df.columns = ['act_R', 'act_D']
df.set_axis(['est_R', 'est_D'], axis = 0)

,act_R,act_D
est_R,2431,3448
est_D,1905,2218


In [50]:
(2431+2218)/(2431+2218+1905+3448)

0.46480703859228156

### Reflections

The classifier got a 46% accuracy, which is similar to the 49% from the speech model. The model made the most mistakes labelling tweets made by Democratic members as  "Republican". The model made the least mistakes labelling Republican-made tweets as Democrat-made tweets. This would seem to indicate that Republicans use language in their tweets that is more distinct to the Republican party than Democrats. However, I would be cautious in drawing any conclusions from this data, given the fact that its accuracy is worse than the flip of a coin.